In [35]:
import numpy as np
from osgeo import ogr, gdal
import rasterio
import json

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path
import re

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

In [37]:
area_cover = '/home/shrayank_mistry/Modules/project-mum-pune/mask.tif'
area = rasterio.open(area_cover, count = 1)

area = np.array(area.read(1))

In [38]:
area_dem = '/home/shrayank_mistry/Modules/project-mum-pune/dem_clipped.tif'
dem = rasterio.open(area_dem, count = 1)

dem = np.array(dem.read(1))
print(dem.shape)

(2721, 3610)


### Up-sampling/Down-sampling

In [39]:
import rasterio
from rasterio.enums import Resampling

# whole-numbers indicate upscaling, fractions indicate downscaling
upscale_factor = 3


with rasterio.open('/home/shrayank_mistry/Modules/project-mum-pune/dem_clipped.tif') as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

In [40]:
area = np.swapaxes(area, 1, 0)
print(area.shape)

(10500, 8273)


In [41]:
up_dem = data[0]
up_dem.shape

(8163, 10830)

In [ ]:
org_dem_map = {}
for i in dem:
    for key in i:
        org_dem_map[key] = org_dem_map.get(key, 0) + 1

up_dem_map = {}

for i in up_dem:
    for key in i:
       up_dem_map[key] = up_dem_map.get(key, 0) + 1 


keys = up_dem_map.keys()

gained_per_class = {}
for k in keys:
    if k in org_dem_map.keys() and k in up_dem_map.keys():
        print((up_dem_map[k] - org_dem_map[k]))
        gained_per_class[k] = gained_per_class.get(k, 0) + (up_dem_map[k] - org_dem_map[k])
    

In [45]:
dem_data, mask_data = up_dem, area
dem_data = dem_data.T


print(dem_data.shape, mask_data.shape)
width, height = 10500, 8163

dem_data, mask_data = dem_data[:width,:height], mask_data[:width, :height]
print(dem_data.shape, mask_data.shape)

(10830, 8163) (10500, 8273)
(10500, 8163) (10500, 8163)


In [46]:
def array2raster(newRasterfn, dataset, array, dtype):
    """
    save GTiff file from numpy.array
    input:
        newRasterfn: save file name
        dataset : original tif file
        array : numpy.array
        dtype: Byte or Float32.
    """
    cols = array.shape[1]
    rows = array.shape[0]
    originX, pixelWidth, b, originY, d, pixelHeight = dataset.GetGeoTransform() 

    driver = gdal.GetDriverByName('GTiff')

    # set data type to save.
    GDT_dtype = gdal.GDT_Unknown
    if dtype == "Byte": 
        GDT_dtype = gdal.GDT_Byte
    elif dtype == "Float32":
        GDT_dtype = gdal.GDT_Float32
    elif dtype == "Int16":
        GDT_dtype = gdal.GDT_Int16
    else:
        print("Not supported data type.")

    # set number of band.
    if array.ndim == 2:
        band_num = 1
    else:
        band_num = array.shape[2]

    outRaster = driver.Create(newRasterfn, cols, rows, band_num, GDT_dtype)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))

    # Loop over all bands.
    for b in range(band_num):
        outband = outRaster.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        if band_num == 1:
            outband.WriteArray(array)
        else:
            outband.WriteArray(array[:,:,b])

    # setteing srs from input tif file.
    prj=dataset.GetProjection()
    outRasterSRS = osr.SpatialReference(wkt=prj)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [47]:
mask_raster = gdal.Open('/home/shrayank_mistry/Modules/project-mum-pune/raster.tif', gdal.GA_ReadOnly)
array2raster('resized-dem.tif', mask_raster, dem_data.T, "Int16")

In [49]:
# r_dem = '/home/shrayank_mistry/Modules/project-mum-pune/resized-dem.tif'
# r_dem = rasterio.open(r_dem, count = 1)

# r_dem = np.array(r_dem.read(1))
# print(r_dem.shape)

(8163, 10500)


In [51]:
# r_dem_data = r_dem.T
# print(mask_data.shape, r_dem_data.shape)

(10500, 8163) (10500, 8163)


In [52]:
print(dem_data.shape, mask_data.shape)

(10500, 8163) (10500, 8163)


### AHP Mapping

In [59]:
ahp_map = {

'urban': 0.29,
'farms': 0.239,
'dense-forest': 0.207,
'water': 0.13,
'fallow': 0.067,
'sparse-forest': 0.049,
'barren-land': 0.019,
'unclassified':0

}

class_map = {

    0: 'unclassified',
    1: 'water',
    2: 'dense-forest',
    3: 'sparse-forest',
    4: 'barren-land',
    5: 'urban',
    6: 'farms',
    7: 'fallow',

}


In [60]:
from copy import deepcopy
mask_copy = deepcopy(mask_data)

In [75]:
def set_weights(c):
    c_str = class_map.get(c)
    return ahp_map[c_str]

In [76]:
set_weights_vtr = np.vectorize(set_weights)
mask_copy = set_weights_vtr(mask_copy)

In [77]:
np.unique(mask_copy, return_counts = True)

(array([0.   , 0.019, 0.049, 0.067, 0.13 , 0.207, 0.239, 0.29 ]),
 array([     588, 19822048, 17097029, 28893954,  7807154,  3858436,
         5940554,  2291737]))

In [78]:
np.unique(mask_data, return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int16),
 array([     588,  7807154,  3858436, 17097029, 19822048,  2291737,
         5940554, 28893954]))

In [81]:
array2raster('weight-mask.tif', mask_raster, mask_copy.T, "Float32")

### Read GeoJson File

In [100]:
import json

path = '/home/shrayank_mistry/Modules/project-mum-pune/main_road.geojson'

with open(path) as f:
    data = json.load(f)

points = []
for feature in data['features']:
    points = feature['geometry']['coordinates']
    print(len(feature['geometry']['coordinates']))

3


In [123]:
pts_arr = np.array(points)
print(pts_arr.shape)

(3,)


### Find Distance

In [103]:
import pandas as pd
import numpy as np
import math
import json
import glob

In [236]:
def deg2rad(deg):
    return (deg * (math.pi/180))

def get_distance(p, q):
    # lat1, lon1 = p
    # lat2, lon2 = q

    lon1, lat1 = p
    lon2, lat2 = q
    #const [lat1, lon1] = pair1, [lat2, lon2] = pair2;
    #const R = 6371; // Radius of the earth in km.

    R = 6371

    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(deg2rad(lat1)) * math.cos(deg2rad(lat2)) * math.sin(dLon/2) * math.sin(dLon/2);
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a));

    return R * c

def find_distance(file_name):
    f = open(file_name)
    data = json.load(f)
 

    points = []
    for feature in data['features']:
        points = feature['geometry']['coordinates']
        print(len(feature['geometry']['coordinates']))
    
    points = np.array(points)

    # print(points[1])

    distance = 0
    # for k in range(len(points)):
    for i in range(1, len(points[1])):
        p = points[1][i - 1]
        q = points[1][i]


        #distance += math.sqrt(math.pow((p[0] - q[0]), 2) + math.pow((p[1] - q[1]), 2))
        distance += get_distance(p, q)
    return distance

In [237]:
index = 0
path = '/home/shrayank_mistry/Modules/project-mum-pune/route-files/'
files = glob.glob('/home/shrayank_mistry/Modules/project-mum-pune/route-files/*.geojson')
# print(files)
for f in files:
    dist = find_distance(f)
    # name = files[index].split('.')[0].title()
    print(f'Distance = {dist} Kms')
    # index = index + 1

3
Distance = 94.45558087050145 Kms


In [233]:
def deg2rad(deg):
    return (deg * (math.pi/180))

def get_distance(p, q):
    lat1, lon1 = p
    lat2, lon2 = q
    #const [lat1, lon1] = pair1, [lat2, lon2] = pair2;
    #const R = 6371; // Radius of the earth in km.

    R = 6371

    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(deg2rad(lat1)) * math.cos(deg2rad(lat2)) * math.sin(dLon/2) * math.sin(dLon/2);
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a));

    return R * c

t_data = None
def find_distance(file_name):
    f = open(file_name)
    data = json.load(f)

    t_data = data
    # print()

    i = 2
    distance = 0
    while (data['elements'][i].get('type') == 'node'):
        p = [data['elements'][i - 1]['lat'], data['elements'][i - 1]['lon']] 
        q = [data['elements'][i]['lat'], data['elements'][i]['lon']] 
        i = i + 1

        distance += get_distance(p, q)
        # print('Works')
    
    # print(i)
 

    # points = []
    # for feature in data['features']:
    #     points = feature['geometry']['coordinates']
    #     print(len(feature['geometry']['coordinates']))
    
    # points = np.array(points)

    # # print(points[1])

    # # for k in range(len(points)):
    # for i in range(1, len(points[1])):
    #     p = points[1][i - 1]
    #     q = points[1][i]


    #     #distance += math.sqrt(math.pow((p[0] - q[0]), 2) + math.pow((p[1] - q[1]), 2))
    #     distance += get_distance(p, q)
    return distance

In [234]:
index = 0
path = '/home/shrayank_mistry/Modules/project-mum-pune/route-files/'
files = glob.glob('/home/shrayank_mistry/Modules/project-mum-pune/route-files/*.json')
# print(files)
for f in files:
    dist = find_distance(f)
    # name = files[index].split('.')[0].title()
    print(f'Distance = {dist} Kms')
    # index = index + 1

Distance = 573.3602491352607 Kms
